In [5]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/esml/common/")
from azureml.core.authentication import InteractiveLoginAuthentication
from esml import ESMLProject

p = ESMLProject()
p.dev_test_prod="dev"

auth = InteractiveLoginAuthentication(tenant_id = p.tenant)
#auth = InteractiveLoginAuthentication(force=True, tenant_id = p.tenant)
ws, config_name = p.authenticate_workspace_and_write_config(auth)

Using lake_settings.json with ESML version 1.4 - Models array support including LABEL


In [6]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
from baselayer_azure_ml_pipeline import ESMLPipelineFactory, esml_pipeline_types

p = ESMLProject()
p.ws = p.get_workspace_from_config()
p.inference_mode = True
p.active_model = 11 # Diabetes
p_factory = ESMLPipelineFactory(p)

scoring_date = '1000-01-01 10:35:01.243860' # 
p_factory.batch_pipeline_parameters[1].default_value = scoring_date # overrides ESMLProject.date_scoring_folder.
p_factory.describe()

Using lake_settings.json with ESML version 1.4 - Models array support including LABEL

 ---- Q: WHICH files are generated as templates, for you to EDIT? ---- 
A: These files & locations:
File to EDIT (step: IN_2_SILVER_1): ../../../2_A_aml_pipeline/4_inference/batch/M11/in2silver_ds01_diabetes.py
File to EDIT (step: IN_2_SILVER_2): ../../../2_A_aml_pipeline/4_inference/batch/M11/in2silver_ds02_other.py
File to EDIT (step: SILVER_MERGED_2_GOLD): ../../../2_A_aml_pipeline/4_inference/batch/M11/silver_merged_2_gold.py
File to EDIT (step: SCORING_GOLD): ../../../2_A_aml_pipeline/4_inference/batch/M11/scoring_gold.py
File to EDIT (step: TRAIN_SPLIT_AND_REGISTER): ../../../2_A_aml_pipeline/4_inference/batch/M11/train_split_and_register.py
File to EDIT (step: TRAIN_MANUAL): ../../../2_A_aml_pipeline/4_inference/batch/M11/train_manual.py
File to EDIT (step: TRAIN_AUTOML): ../../../2_A_aml_pipeline/4_inference/batch/M11/train_post_automl_step.py
File to EDIT a lot (reference in step-scripts Cus

In [ ]:
# CREATE - python files in snapshot folder
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=True) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT

In [ ]:
## BUILD
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD) # Creates pipeline from template

# Smoke test - see if pipeline works? 

In [ ]:
## RUN
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

## Look at the scored data

In [7]:
from azureml.core.dataset import Dataset

name = p.ModelAlias + '_GOLD_TO_SCORE'
# Look at scored result
ds07 = Dataset.get_by_name(workspace=p.ws, name=name,  version='latest') 
ds07.to_pandas_dataframe().head()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641


# Publish - To use from REST or from Azure Data factory

In [ ]:
# PUBLISH
published_pipeline, endpoint = p_factory.publish_pipeline(batch_pipeline,"_1") # "_1" is optional    to create a NEW pipeline with 0 history, not ADD version to existing pipe & endpoint

### Get the Piplien ID to user in Azure ML Activity from Azure DATA FACTORY

In [ ]:
print("2) Fetch scored data: Below needed for Azure Data factory PIPELINE activity (Pipeline OR Endpoint. Choose the latter") 
print ("- Endpoint ID")
print("Endpoint ID:  {}".format(endpoint.id))
print("Endpoint Name:  {}".format(endpoint.name))
print("Experiment name:  {}".format(p_factory.experiment_name))

print("In AZURE DATA FACTORY - This is the ID you need, if using PRIVATE LINK, private Azure ML workspace.")
print("-You need PIPELINE id, not pipeline ENDPOINT ID ( since cannot be chosen in Azure data factory if private Azure ML)")
published_pipeline.id

# END

## MISC: Other curated - change environment

In [ ]:

p_factory.use_curated_automl_environment = True
p_factory.environment_name = "AzureML-AutoML-DNN" # Training[ "AzureML-AutoML", "AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu"]  Inference["AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference",]

## BUILD
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=False) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(pipeline_type=esml_pipeline_types.IN_2_GOLD_SCORING,same_compute_for_all=True)

## Non-curated - custom conda/pip definition

In [ ]:
p_factory.use_curated_automl_environment = False

######### See here for environments: https://docs.microsoft.com/en-us/azure/machine-learning/resource-curated-environments

######### ESML Defaults to the below CONDA, when use_curated_automl_environment = False 
#aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
    #conda_packages=['pandas','scikit-learn'], 
    #pip_packages=['azureml-sdk[automl]', 'pyarrow'])

## BUILD
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=False) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(pipeline_type=esml_pipeline_types.IN_2_GOLD_SCORING,same_compute_for_all=True)